In [1]:

!pip install pdfplumber
!pip install accelerate==0.26.0
!pip install --upgrade transformers
!pip install -U bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 105.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 78.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
    Uninstalling accelerate-1.2.1:
      Successfully uninstalled accelerate-1.2.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 33.4 MB/s eta 0:00:0000:010:01m
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0

In [ ]:
import torch
import io
import pandas as pd
from tqdm import tqdm 
import pdfplumber
device = "cuda" if torch.cuda.is_available() else "cpu"
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline,BitsAndBytesConfig
torch.random.manual_seed(0)
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct", device_map="cuda", torch_dtype="auto",quantization_config=BitsAndBytesConfig(load_in_4bit=True), trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

def extract_pdf_content(pdf_path):
        with pdfplumber.open(pdf_path) as pdf:
            text = ""
            # Loop through each page in the PDF
            for page in pdf.pages:
                text += page.extract_text()  # Extract text from the page
        return text



def phi_3(resume_text,job_description,model=model,tokenizer=tokenizer):
    messages = [
    {
        "role": "system",
        "content": "You are an AI designed to analyze resumes and compare them with job descriptions. Your task is to determine how well a given resume matches a job description by calculating a match percentage, identifying key skills mentioned in the JD that are present or missing in the resume, and highlighting any gaps such as experience level mismatches. Provide a structured and detailed response, including both positive and negative aspects of the match."
    },
    {
        "role": "user",
        "content": "I will provide a resume text and a job description. Your task is to evaluate the match percentage, list the matched and missing key skills, highlight any mismatches (such as experience, education, or domain expertise), and provide a structured assessment explaining how well the resume aligns with the job description."
    },
    {
        "role": "assistant",
        "content": "Understood. I will analyze the resume against the job description, calculate the match percentage, identify key skills that match or are missing, and highlight mismatches such as experience level, required certifications, or specific industry/domain expertise. I will then provide a clear and structured assessment, including both strengths and weaknesses."
    },
    {"role": "user", "content": resume_text},  # Replace this with the actual resume text
    {"role": "user", "content": job_description}  # Replace this with the actual job description
    ]



    generation_args = {
    "max_new_tokens": 1500,
    "do_sample": True,
    "temperature": 0.7,
    "top_k": 50,
    "top_p": 0.9,
    "use_cache": False  # Disable caching to avoid DynamicCache issues
}
    
    output = pipe(messages, **generation_args)
    return output[0]['generated_text']

    
total_co=''
pdf_path=input('enter the pdf path :____')
pdf_m=extract_pdf_content(pdf_path)
que = input('paste the job discription____ ')
total_co=phi_3(resume_text=pdf_m,job_description=que)
print(total_co)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


enter the pdf path :____ /kaggle/input/my-resume/N.surya_narayanan_resume_MscDs_.pdf (1) (1).pdf
paste the job discription____  Job Title: Data Scientist (Deep Learning & Generative AI) 📍 Location: Chennai, Noida 📄 Job Type: Full-time 💼 Experience: 2-3 years  About the Role: We are seeking a Data Scientist specializing in Deep Learning and Generative AI with 2-3 years of experience to join our AI-driven team. This role is ideal for someone passionate about Neural Networks, Transformers, Large Language Models (LLMs), and NLP. You will be responsible for building, fine-tuning, and deploying AI models using Hugging Face, TensorFlow, PyTorch, and OpenAI APIs.  Responsibilities: ✔️ Develop and fine-tune Deep Learning models for NLP, CV, and Generative AI tasks. ✔️ Implement and optimize Transformer-based models (e.g., GPT, BERT, T5, LLaMA, Mistral). ✔️ Work with Hugging Face’s Transformers, Tokenizers, and Datasets for model training and deployment. ✔️ Design LLM-powered applications using 

In [2]:
import torch
import pdfplumber
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

device = "cuda" if torch.cuda.is_available() else "cpu"

torch.random.manual_seed(0)
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct", 
    device_map=device, 
    torch_dtype="auto",
    # quantization_config=BitsAndBytesConfig(load_in_4bit=True), 
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

def extract_pdf_content(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = "".join(page.extract_text() or "" for page in pdf.pages)
    return text

def phi_3_stream(resume_text, job_description):
    messages = [
    {
        "role": "system",
        "content": "You are an AI designed to analyze resumes and compare them with job descriptions. Your task is to determine how well a given resume matches a job description by calculating a match percentage, identifying key skills mentioned in the JD that are present or missing in the resume, and highlighting any gaps such as experience level mismatches. Provide a structured and detailed response, including both positive and negative aspects of the match."
    },
    {
        "role": "user",
        "content": "I will provide a resume text and a job description. Your task is to evaluate the match percentage, list the matched and missing key skills, highlight any mismatches (such as experience, education, or domain expertise), and provide a structured assessment explaining how well the resume aligns with the job description."
    },
    {
        "role": "assistant",
        "content": "Understood. I will analyze the resume against the job description, calculate the match percentage, identify key skills that match or are missing, and highlight mismatches such as experience level, required certifications, or specific industry/domain expertise. I will then provide a clear and structured assessment, including both strengths and weaknesses."
    },
    {"role": "user", "content": resume_text},  # Replace this with the actual resume text
    {"role": "user", "content": job_description}  # Replace this with the actual job description
    ]
    
    input_text = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
    
    generation_args = {
        "max_new_tokens": 1500,
        "do_sample": True,
        "temperature": 0.7,
        "top_k": 50,
        "top_p": 0.9
    }
    
    print("\nAI Response:")
    with torch.no_grad():
        output_ids = model.generate(input_text, **generation_args)
        for token_id in output_ids[:, input_text.shape[-1]:][0]:
            print(tokenizer.decode([token_id], skip_special_tokens=True), end="", flush=True)

pdf_path = input('Enter the PDF path: ')
pdf_text = extract_pdf_content(pdf_path)
job_description = input('Paste the job description: ')
phi_3_stream(resume_text=pdf_text, job_description=job_description)

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Enter the PDF path:  /kaggle/input/my-resume/N.surya_narayanan_resume_MscDs_.pdf (1) (1).pdf
Paste the job description:  Job Title: Data Scientist (Deep Learning & Generative AI) 📍 Location: Chennai, Noida 📄 Job Type: Full-time 💼 Experience: 2-3 years  About the Role: We are seeking a Data Scientist specializing in Deep Learning and Generative AI with 2-3 years of experience to join our AI-driven team. This role is ideal for someone passionate about Neural Networks, Transformers, Large Language Models (LLMs), and NLP. You will be responsible for building, fine-tuning, and deploying AI models using Hugging Face, TensorFlow, PyTorch, and OpenAI APIs.  Responsibilities: ✔️ Develop and fine-tune Deep Learning models for NLP, CV, and Generative AI tasks. ✔️ Implement and optimize Transformer-based models (e.g., GPT, BERT, T5, LLaMA, Mistral). ✔️ Work with Hugging Face’s Transformers, Tokenizers, and Datasets for model training and deployment. ✔️ Design LLM-powered applications using LangCha

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



AI Response:


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


AttributeError: 'DynamicCache' object has no attribute 'get_max_length'

In [ ]:
# /kaggle/input/my-resume/N.surya_narayanan_resume_MscDs_.pdf (1) (1).pdf
Job Title: Data Scientist (Deep Learning & Generative AI)
📍 Location: Chennai, Noida
📄 Job Type: Full-time
💼 Experience: 2-3 years

About the Role:
We are seeking a Data Scientist specializing in Deep Learning and Generative AI with 2-3 years of experience to join our AI-driven team. This role is ideal for someone passionate about Neural Networks, Transformers, Large Language Models (LLMs), and NLP. You will be responsible for building, fine-tuning, and deploying AI models using Hugging Face, TensorFlow, PyTorch, and OpenAI APIs.

Responsibilities:
✔️ Develop and fine-tune Deep Learning models for NLP, CV, and Generative AI tasks.
✔️ Implement and optimize Transformer-based models (e.g., GPT, BERT, T5, LLaMA, Mistral).
✔️ Work with Hugging Face’s Transformers, Tokenizers, and Datasets for model training and deployment.
✔️ Design LLM-powered applications using LangChain, RAG, and Prompt Engineering techniques.
✔️ Handle data preprocessing, augmentation, and embedding techniques (FAISS, ChromaDB, Pinecone, Weaviate).
✔️ Train and optimize Diffusion Models (Stable Diffusion, DALL·E) and GANs for image generation tasks.
✔️ Work with cloud platforms (AWS, GCP, Azure) for scalable model training and deployment.
✔️ Deploy AI models using FastAPI, Flask, Docker, and Kubernetes in production environments.
✔️ Collaborate with cross-functional teams to build AI-powered applications and solutions.
✔️ Stay updated with cutting-edge advancements in AI, Deep Learning, and Generative AI.

Requirements:
🎓 Bachelor’s/Master’s degree in Computer Science, AI, Data Science, or related fields.
💻 2-3 years of hands-on experience in Deep Learning, NLP, or Generative AI projects.
🐍 Strong Python skills (NumPy, Pandas, PyTorch, TensorFlow, Hugging Face Transformers).
🧠 Solid understanding of Neural Networks, Attention Mechanisms, and Transfer Learning.
🔍 Experience with LLMs, RAG, and model fine-tuning using Hugging Face or OpenAI APIs.
📊 Proficiency in vector databases (FAISS, ChromaDB, Pinecone) for embedding search.
☁️ Hands-on experience with Cloud AI services (AWS SageMaker, GCP Vertex AI, Azure AI).
🛠️ Experience with MLOps tools (MLflow, Kubeflow, Docker, Kubernetes, CI/CD pipelines).
📈 Strong grasp of Deep Learning frameworks (PyTorch Lightning, TensorFlow/Keras).
💡 Problem-solving mindset and ability to work on real-world AI challenges.

Nice to Have:
✅ Experience with Diffusion Models (Stable Diffusion, DALL·E, Midjourney).
✅ Knowledge of Graph Neural Networks (GNNs) and Reinforcement Learning (RLHF).
✅ Exposure to AI Ethics, Bias Mitigation, and Explainable AI (XAI).
✅ Contributions to open-source AI projects or research publications.

🚀 Join us and work on cutting-edge AI, Deep Learning, and Generative AI models!

In [ ]:
ob Title: Data Science 

Location: chennai,noida
Job Type: Full-time
Experience: 2-3 year

About the Role:
We are looking for a highly motivated Data Science Fresher to join our team. This role is ideal for someone passionate about data analysis, machine learning, and AI-driven solutions. You will work with large datasets, develop predictive models, and gain hands-on experience with cutting-edge tools and technologies.

Responsibilities:
Collect, clean, and preprocess structured and unstructured data.
Perform exploratory data analysis (EDA) to uncover insights.
Develop and implement machine learning models for predictive analytics.
Work with tools like Python, SQL, and cloud platforms for data processing.
Collaborate with cross-functional teams to solve real-world business problems.
Create dashboards and reports using visualization tools (Tableau, Power BI, Matplotlib, etc.).
Stay updated with the latest trends and advancements in data science.
Requirements:
Bachelor’s/Master’s degree in Computer Science, Data Science, Mathematics, or a related field.
Strong programming skills in Python (NumPy, Pandas, Scikit-Learn) or R.
Knowledge of SQL for data extraction and manipulation.
Understanding of machine learning concepts and algorithms.
Familiarity with data visualization tools like Matplotlib, Seaborn, or Power BI.
Basic knowledge of cloud platforms (AWS, Azure, or GCP) is a plus.
Strong problem-solving skills and attention to detail.
Good communication skills and a willingness to learn.
Nice to Have:
Experience with NLP, Deep Learning, or Big Data technologies.
Hands-on projects or internships in data science.
Familiarity with tools like Git, Docker, or MLOps frameworks.